In [1]:
%matplotlib inline
import pandas as pd
import d2l.torch as d2l
import torch
from torch.nn import functional as F
from torch import nn
import numpy as np
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from pandas.api.types import is_numeric_dtype
import time
from sklearn.preprocessing import StandardScaler, MinMaxScaler

torch.manual_seed(0)

In [ ]:
dfs = pd.DataFrame()
for i in range(1, 6):
    df = pd.read_excel(f"./data/flight_data_batch{i}.xlsx")
    dfs = pd.concat([dfs, df], ignore_index=True)

In [ ]:
for i in range(1, 4):
    df = pd.read_excel(f"./data/flight_data_batch6_part{i}.xlsx")
    dfs = pd.concat([dfs, df], ignore_index=True)

In [ ]:
# only visualizing a portion of data
#all_data = df.iloc[:9000]


all_data = dfs.groupby("label")

normal_groups = all_data.get_group(0)
bad_groups = all_data.get_group(1)    

In [ ]:
# compare normal vs bad flights
normal_flights = normal_groups.groupby("flight_id")
normal_flights = [f for _, f in normal_flights]
bad_flights = bad_groups.groupby("flight_id")
bad_flights = [f for _, f in bad_flights]

In [2]:
t = normal_data["time"].iloc[:200] # time axis
normal_data = normal_data.drop("time", axis=1) # drop the time column
bad_data = bad_data.drop("time", axis=1)

# standard norm for all columns
normal_data_scaled = normal_data.transform(lambda col : (col - col.mean()) / (col.std() + 1e-9) if is_numeric_dtype(col) else col)
bad_data_scaled = bad_data.transform(lambda col : (col - col.mean()) / (col.std() + 1e-9) if is_numeric_dtype(col) else col)

# Set general font size for matplot
plt.rcParams['font.size'] = '16'

num_graphs = normal_data_scaled.shape[1] - 1
decomped = []
for i in range(num_graphs-1):
    fig, axes = plt.subplots(4, 2, figsize=(10, 15))
        
    column_name = normal_data_scaled.columns[i]
    axes[0, 0].set_ylabel(column_name)
    axes[0, 0].plot(t, normal_data_scaled[column_name])
    axes[0, 0].plot(t, [normal_data_scaled[column_name].mean()]*len(t), color="red")
    axes[0, 0].legend(["Measured", "Fit"])
    # STL decomp
    stl = STL(normal_data_scaled[column_name], period=10)
    res = stl.fit()
    axes[1, 0].set_ylabel("Trend")
    axes[1, 0].plot(t, res.trend)
    axes[2, 0].set_ylabel("Seasonal")
    axes[2, 0].plot(t, res.seasonal)
    axes[3, 0].set_ylabel("Residual")
    axes[3, 0].plot(t, res.resid)
    axes[3, 0].set_xlabel("time")
    # decomped.append(res)
        
    column_name = bad_data_scaled.columns[i]
    axes[0, 1].set_ylabel(column_name)
    axes[0, 1].plot(t, bad_data_scaled[column_name], color="orange")
    axes[0, 1].plot(t, [bad_data_scaled[column_name].mean()]*len(t), color="black")
    axes[0, 1].legend(["Measured", "Fit"])
    # STL decomp
    stl = STL(bad_data_scaled[column_name], period=10)
    res = stl.fit()
    axes[1, 1].plot(t, res.trend, color="orange")
    axes[2, 1].plot(t, res.seasonal, color="orange")
    axes[3, 1].plot(t, res.resid, color="orange")
    axes[3, 1].set_xlabel("time")
    # Add a horizontal line divider below to separate every 4 graphs 
    fig.add_artist(plt.Line2D((0, 1), (0.08, 0.08), color='black', 
                  linewidth=5, transform=fig.transFigure))
    plt.subplots_adjust(bottom=0.15)

    


NameError: name 'normal_data' is not defined